In [1]:
# infer GPU in use
!nvidia-smi

Wed Jul 17 12:17:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   20C    P0              51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import numpy as np
import os, sys
import matplotlib.pyplot as plt

# in this notebook, we will analyse the data files and try to see how we can make a custom data loader for the same.

data_y_s = np.load('/lcrc/project/NEXTGENOPT/NREL_COMSTOCK_DATA/grouped/G4601010_data.npz')
data_x_u = np.load('/lcrc/project/NEXTGENOPT/NREL_COMSTOCK_DATA/grouped/G4601010_weather.npz')

# function to calculate model sizes

def model_size_in_mb(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    size_in_mb = param_size / (1024 ** 2)
    return size_in_mb

In [3]:
# here we write some basic code which allows the creation of a dataloader

import torch
from torch.utils.data import Dataset
from typing import Union, List, Tuple
from itertools import combinations

# class LFDataset(Dataset):
    
#     def __init__(
#         self,
#         data_y_s: np.array,
#         data_x_u: np.array,
#         lookback: int,
#         lookahead: int,
#         client_idx: int,
#         idx_x: Union[List,Tuple],
#         idx_u: Union[List,Tuple],
#         dtype: torch.dtype = torch.float32
#     ):
        
#         # sanity checks
#         assert lookback > 0, "Cannot have non-positive lookback!"
#         assert lookahead > 0, "Cannot have non-positive lookahead!"
#         assert client_idx < data_y_s['load'].shape[0], "Client index exceeds number of clients present."
#         assert len(idx_x)+len(idx_u) == data_x_u['wdata'].shape[0], "Indices provided do not sum upto the input dimension."
#         assert all(not set(a) & set(b) for a, b in combinations([idx_x, idx_u], 2)), "All indices are not mutually exclusive."
        
#         # save inputs
#         self.load = data_y_s['load'][client_idx,:]
#         self.static = data_y_s['static'][client_idx,:]
#         self.x, self.u = data_x_u['wdata'][idx_x,:], data_x_u['wdata'][idx_u,:]
#         self.idx_x, idx_u = idx_x, idx_u
#         self.lookback, self.lookahead = lookback, lookahead
#         self.dtype = dtype
        
#         # max length
#         self.maxlen = self.load.shape[0] - lookback - lookahead + 1
        
#     def __len__(self):
        
#         return self.maxlen
    
#     def __getitem__(self, idx):
        
#         y_past = torch.tensor(self.load[idx:idx+self.lookback][:,None], dtype=self.dtype)
#         x_past = torch.tensor(self.x[:,idx:idx+self.lookback].T, dtype=self.dtype)
#         u_past = torch.tensor(self.u[:,idx:idx+self.lookback].T, dtype=self.dtype)
#         u_future = torch.tensor(self.u[:,idx+self.lookback:idx+self.lookback+self.lookahead].T, dtype=self.dtype)
#         s_past = torch.tensor(self.static[None,:].repeat(self.lookback,axis=0), dtype=self.dtype)
#         y_target = torch.tensor(self.load[idx+self.lookback+self.lookahead-1].reshape((1,)), dtype=self.dtype)
#         y_all_target = torch.tensor(self.load[idx+self.lookback:idx+self.lookback+self.lookahead][:,None], dtype=self.dtype)
        
#         inp = (y_past,x_past,u_past,s_past,u_future)
#         lab = (y_target, y_all_target)
        
#         return inp, lab

In [4]:
# We now test out our dataset with california data

sys.path.insert(0,'/home/sbose/time-series-forecasting-federation')
from models.LFDataset import LFDataset

# create dataset
CA_dset = LFDataset(
    data_y_s = data_y_s,
    data_x_u = data_x_u,
    lookback = 8,
    lookahead = 4,
    client_idx = 0,
    idx_x = [0,1,2,3,4,5],
    idx_u = [6,7],
    dtype = torch.float32
)

# load into dataloader
from torch.utils.data import DataLoader
CA_dataloader = DataLoader(CA_dset, batch_size = 32, shuffle = True)

In [5]:
# Test out the shape of the dataloader outputs

for cidx, (a,b) in enumerate(CA_dataloader):
    print(f"On {cidx+1}th item of dataloader, type of a is {type(a)}, type of b is {type(b)}")
    for idx,itm in enumerate(a):
        print(f"Shape of {idx+1}th item in a is {itm.shape}.")
    for idx,itm in enumerate(b):
        print(f"Shape of {idx+1}th item in b is {itm.shape}.")
    break

On 1th item of dataloader, type of a is <class 'models.LFDataset.TensorList'>, type of b is <class 'models.LFDataset.TensorList'>
Shape of 1th item in a is torch.Size([32, 8, 1]).
Shape of 2th item in a is torch.Size([32, 8, 6]).
Shape of 3th item in a is torch.Size([32, 8, 2]).
Shape of 4th item in a is torch.Size([32, 8, 7]).
Shape of 5th item in a is torch.Size([32, 4, 2]).
Shape of 6th item in a is torch.Size([32, 4, 1]).
Shape of 1th item in b is torch.Size([32, 1]).
Shape of 2th item in b is torch.Size([32, 4, 1]).


In [6]:
# Ensure that relative imports from the git repository can always be found

import sys
sys.path.insert(0,'/home/sbose/time-series-forecasting-federation')

In [7]:
# test out LSTM vanilla version

import torch
import torch.nn as nn
from models.LSTM.LSTMFCDecoder import LSTMFCDecoder

model = LSTMFCDecoder(
    input_size = 16,
    hidden_size = 20,
    num_layers = 2,
    y_size = 1,
    fcnn_sizes = (160,80,10,1),
    activation = nn.ReLU,
    lookback = 8,
    lookahead = 4,
    dtype = torch.float32
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of LSTM output is {tuple(w.shape)}.")
    break

# print model size
print(f"LSTM FCNN head: {model_size_in_mb(model)} MB")

Shape of LSTM output is (32, 1).
LSTM FCNN head: 0.07667922973632812 MB


In [8]:
# test out LSTM autoregressive version

import torch
import torch.nn as nn
from models.LSTM.LSTMAR import LSTMAR

model = LSTMAR(
    input_size = 16, # x + u + y + s
    u_size = 2, # u
    hidden_size = 20,
    num_layers = 2,
    y_size = 1, # y
    fcnn_sizes = (20,10,10,1),
    activation = nn.ReLU,
    lookahead = 4,
    dtype = torch.float32
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of LSTM output is {tuple(w.shape)}.")
    break

# print model size
print(f"LSTM AR: {model_size_in_mb(model)} MB")

Shape of LSTM output is (32, 4, 1).
LSTM AR: 0.045818328857421875 MB


In [9]:
# import os
# import torch
# import numpy as np

# # we first map data types here

# dtype_map = {
#     torch.float32: np.float32,
#     torch.float: np.float32,
#     torch.float64: np.float64,
#     torch.double: np.float64,
#     torch.float16: np.float16,
#     torch.half: np.float16,
#     torch.uint8: np.uint8,
#     torch.int8: np.int8,
#     torch.int16: np.int16,
#     torch.short: np.int16,
#     torch.int32: np.int32,
#     torch.int: np.int32,
#     torch.int64: np.int64,
#     torch.long: np.int64,
#     torch.bool: np.bool_,
#     torch.complex64: np.complex64,
#     torch.complex128: np.complex128
# }

# # some functions to flatten and unflatten state dicts to.from numpy vectors

# def state_dict_to_vector(state_dict):
#     param_vector = [param.detach().cpu().numpy().astype(dtype_map[param.dtype]).flatten() for param in state_dict.values()]
#     return np.concatenate(param_vector)

# def load_vector_to_state_dict(vector, reference_state_dict):
#     pointer = 0
#     with torch.no_grad():
#         for param in reference_state_dict.values():
#             num_param = param.numel()
#             param.copy_(torch.from_numpy(vector[pointer:pointer + num_param]).view_as(param).to(param.dtype))
#             pointer += num_param

# def gradients_to_vector(state_dict, dtype_map):
#     grad_vector = [(param.grad.detach().cpu().numpy().astype(dtype_map[param.dtype]).flatten() if param.grad is not None
#                     else np.zeros(param.numel(), dtype=dtype_map[param.dtype]))
#                    for param in state_dict.values()]
#     return np.concatenate(grad_vector)

# def load_vector_to_gradients(vector, reference_state_dict, dtype_map):
#     pointer = 0
#     with torch.no_grad():
#         for param in reference_state_dict.values():
#             num_elements = param.numel()
#             part_vector = vector[pointer:pointer + num_elements]
#             grad_tensor = torch.from_numpy(part_vector).view_as(param).to(dtype=dtype_map[param.dtype])
#             if param.grad is not None:
#                 param.grad.copy_(grad_tensor)
#             else:
#                 param.grad = grad_tensor.to(param.device)
#             pointer += num_elements
            
# # we now load the name of all the files

# base_grouped_dir = '/lcrc/project/NEXTGENOPT/NREL_COMSTOCK_DATA/grouped'
# suffix = '_data.npz'
# filenames = []

# for filename in os.listdir(base_grouped_dir):
#     if filename.endswith(suffix):
#         # Extract the leading characters and add them to the list
#         filenames.append(filename[:-len(suffix)])



In [10]:
# test out DARNN

import torch
import torch.nn as nn
from models.DARNN.DARNN import DARNN

model = DARNN(
        x_size = 6,
        y_size = 1,
        u_size = 2,
        s_size = 7,
        encoder_hidden_size = 20,
        decoder_hidden_size = 20,
        encoder_num_layers = 2,
        decoder_num_layers = 2,
        lookback = 8,
        lookahead = 4,
        dtype = torch.float32
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of LSTM output is {tuple(w.shape)}.")
    break

# print model size
print(f"DARNN: {model_size_in_mb(model)} MB")

Shape of LSTM output is (32, 4, 1).
DARNN: 0.0562744140625 MB


In [11]:
# test out DARNN

import torch
import torch.nn as nn
from models.TRANSFORMER.TransformerAR import TransformerAR

model = TransformerAR(
        x_size = 6,
        y_size = 1,
        u_size = 2,
        s_size = 7
)

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    wtest = model(a,mode='test')
    print(f"Shape of Transformer AR output is {tuple(w.shape)}.")
    print(f"Shape of Transformer AR output in test mode is {tuple(wtest.shape)}.")
    break

# print model size
print(f"Transformer AR: {model_size_in_mb(model)} MB")

Shape of Transformer AR output is (32, 4, 1).
Shape of Transformer AR output in test mode is (32, 4, 1).
Transformer AR: 4.423839569091797 MB
